Thanks to the following notebooks I used as references

https://www.kaggle.com/archanabenur/lightgbm-and-catboost-prediction

https://www.kaggle.com/arunamenon/stock-price-jane-street-forecasting/output

https://www.kaggle.com/kamalnaithani/lightgbm-stock-prediction-1-1


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance

In [ ]:
train_data = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")
feature_data = pd.read_csv("/kaggle/input/jane-street-market-prediction/features.csv")
example_test =  pd.read_csv("/kaggle/input/jane-street-market-prediction/example_test.csv")

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
train_data.isnull().sum()

In [ ]:
feature_data.head()

In [ ]:
example_test.head()

In [ ]:
resps = train_data[['resp_1','resp_2','resp_3','resp_4','resp']]
pd.plotting.scatter_matrix(resps, figsize=(6, 6))
plt.show()

In [ ]:
# train_data_noN = train_data.dropna()

train_data.fillna(0,inplace=True)
train_data_noN = train_data
train_data = train_data_noN[train_data_noN['weight'] != 0]
print(train_data.shape)

select some features, Reference
[Jane Day 242 Feature Generation and Selection](https://www.kaggle.com/rajkumarl/jane-day-242-feature-generation-and-selection)

In [ ]:
feature_names = ['feature_'+str(i) for i in range(130)]

# feature_names = [ 'feature_1', 'feature_2', 'feature_6', 'feature_9',
#        'feature_10', 'feature_16', 'feature_20', 'feature_29', 'feature_37',
#        'feature_38', 'feature_39', 'feature_40', 'feature_51', 'feature_52',
#        'feature_53', 'feature_54', 'feature_69', 'feature_70', 'feature_71',
#        'feature_83', 'feature_100', 'feature_109', 'feature_112',
#        'feature_122', 'feature_123', 'feature_124', 'feature_126',
#        'feature_128', 'feature_129','weight']
print(len(feature_names))

In [ ]:

train_data['action'] = (train_data_noN['resp'] > 0).astype('int') 

# X = train_data_noN.drop(['resp_1','resp_2','resp_3','resp_4','resp','weight','date','ts_id'],axis=1)

# Y = train_data.loc[:, 'action']
Y = train_data.loc[:, 'resp']
X= train_data.loc[:,feature_names]
print(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=15)

In [ ]:
# param = {'max_depth': 5, 'eta': 0.1, 'objective': 'binary:logistic','eval_metric':'auc'}
param = {'max_depth': 5, 'eta': 0.1}
num_round =500

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round,evallist,early_stopping_rounds=30, verbose_eval= 50)
bst.save_model('xgb2.model')

In [ ]:
# fig,ax = plt.subplots(figsize=(25,50))
# lgbm.plot_importance(bst, ax=ax,importance_type='gain',max_num_features=130)
# plt.show()
plot_importance(bst)
plt.show()

In [ ]:

bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('xgb2.model')  # load data

In [ ]:
to_action = lambda t: 1 * (t > 0)
    

In [ ]:
test_data =  example_test.loc[:, feature_names]
# test_data.head()
test_matrix = xgb.DMatrix(test_data )

y_preds = bst.predict(test_matrix)
print(y_preds)
y_preds = to_action(bst.predict(test_matrix))
print(y_preds)

In [ ]:
import janestreet
env = janestreet.make_env() # ini tialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set


for (test_df, sample_prediction_df) in iter_test:
    if test_df['weight'].item() > 0:
        test = test_df.loc[:, feature_names]
        test = test.fillna(0)
        #         print(X_test.shape)
        test_matrix = xgb.DMatrix(test)
        y_preds = to_action(bst.predict(test_matrix))
        sample_prediction_df.action = y_preds.astype(int)  #make your 0/1 prediction here
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)